# ЭКСПЕРИМЕНТЫ: K-MEANS UNDERSAMPLING ДЛЯ БАЛАНСИРОВКИ КЛАССОВ

═══════════════════════════════════════════════════════════════════════════════

**ЦЕЛЬ:** Исследовать эффективность кастомного K-Means undersampling для борьбы с дисбалансом классов

**ПОДХОД:**
- Кластеризация класса 0 (не ушедшие) через MiniBatchKMeans
- Умный sampling: маленькие кластеры берем полностью, из больших - пропорционально
- Целевое соотношение: 1:20 (вместо исходного ~1:65)
- Обучение 3 алгоритмов: CatBoost, LightGBM, XGBoost

**КОНТЕНТ:**
1. Загрузка и препроцессинг (из Churn_Model_Complete.ipynb)
2. Визуализация распределений всех признаков
3. Кастомный K-Means undersampling (30 кластеров)
4. Обучение моделей на balanced данных
5. Сравнение результатов

═══════════════════════════════════════════════════════════════════════════════

---
## 1. ИМПОРТ БИБЛИОТЕК И КОНФИГУРАЦИЯ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ИМПОРТ БИБЛИОТЕК
# ═══════════════════════════════════════════════════════════════════════════════

import os
import sys
import warnings
from datetime import datetime
from pathlib import Path
import json
import time
import gc

# Данные
import numpy as np
import pandas as pd
from scipy import stats

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# ML - Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.cluster import MiniBatchKMeans

# ML - Models
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
import xgboost as xgb

# ML - Metrics
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve,
    classification_report, confusion_matrix,
    accuracy_score, f1_score, precision_score, recall_score
)

# Настройки
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("═"*80)
print("ЭКСПЕРИМЕНТЫ: K-MEANS UNDERSAMPLING")
print("═"*80)
print(f"✓ Библиотеки импортированы")
print(f"  Дата запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("═"*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# КОНФИГУРАЦИЯ
# ═══════════════════════════════════════════════════════════════════════════════

# Воспроизводимость
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Пути
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output")
MODEL_DIR = Path("models")
FIGURES_DIR = Path("figures")

# Создание папок
for dir_path in [OUTPUT_DIR, MODEL_DIR, FIGURES_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Файлы
TRAIN_FILE = "churn_train_ul.parquet"

# Колонки
ID_COLUMNS = ['cli_code', 'client_id', 'observation_point']
TARGET_COLUMN = 'target_churn_3m'
SEGMENT_COLUMN = 'segment_group'
DATE_COLUMN = 'observation_point'
CATEGORICAL_FEATURES = ['segment_group', 'obs_month', 'obs_quarter']

# Сегменты
SEGMENT_1_VALUES = ['SMALL_BUSINESS']
SEGMENT_2_VALUES = ['MIDDLE_BUSINESS', 'LARGE_BUSINESS']

# Временное разбиение
TRAIN_SIZE = 0.70
VAL_SIZE = 0.15
TEST_SIZE = 0.15

# K-Means параметры
N_CLUSTERS = 30
TARGET_RATIO = 20  # Целевое соотношение class_0 / class_1
KMEANS_BATCH_SIZE = 10000

# Preprocessing
CORRELATION_THRESHOLD = 0.85
OUTLIER_IQR_MULTIPLIER = 1.5

print("\n✓ Конфигурация установлена")
print(f"  Random seed: {RANDOM_SEED}")
print(f"  K-Means кластеров: {N_CLUSTERS}")
print(f"  Целевое соотношение: 1:{TARGET_RATIO}")
print(f"  Segment 1: {SEGMENT_1_VALUES}")
print(f"  Segment 2: {SEGMENT_2_VALUES}")

---
## 2. ЗАГРУЗКА ДАННЫХ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ЗАГРУЗКА PARQUET
# ═══════════════════════════════════════════════════════════════════════════════

train_path = DATA_DIR / TRAIN_FILE

print("\n" + "═"*80)
print("ЗАГРУЗКА ДАННЫХ")
print("═"*80)
print(f"Файл: {train_path}")

if not train_path.exists():
    raise FileNotFoundError(f"Файл не найден: {train_path}")

start = time.time()
df_full = pd.read_parquet(train_path)
load_time = time.time() - start

print(f"\n✓ Загружено за {load_time:.2f} сек")
print(f"  Размер: {df_full.shape}")
print(f"  Память: {df_full.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

# Целевая переменная
churn_rate = df_full[TARGET_COLUMN].mean()
print(f"\n  Target '{TARGET_COLUMN}':")
print(f"    Churn rate: {churn_rate:.4f} ({churn_rate*100:.2f}%)")
print(f"    Churned: {df_full[TARGET_COLUMN].sum():,}")
print(f"    Ratio: 1:{(1-churn_rate)/churn_rate:.1f}")

# Временной диапазон
df_full[DATE_COLUMN] = pd.to_datetime(df_full[DATE_COLUMN])
print(f"\n  Период: {df_full[DATE_COLUMN].min().date()} - {df_full[DATE_COLUMN].max().date()}")
print(f"  Уникальных дат: {df_full[DATE_COLUMN].nunique()}")

print("═"*80)

---
## 3. ВРЕМЕННОЕ РАЗБИЕНИЕ (TRAIN / VAL / TEST-OOT)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# TEMPORAL SPLIT
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("ВРЕМЕННОЕ РАЗБИЕНИЕ (TEMPORAL SPLIT)")
print("═"*80)

# Сортировка по времени
df_sorted = df_full.sort_values(DATE_COLUMN).reset_index(drop=True)
unique_dates = sorted(df_sorted[DATE_COLUMN].unique())
n_dates = len(unique_dates)

# Cutoff indices
train_cutoff = int(n_dates * TRAIN_SIZE)
val_cutoff = int(n_dates * (TRAIN_SIZE + VAL_SIZE))

train_end = unique_dates[train_cutoff - 1]
val_end = unique_dates[val_cutoff - 1]

print(f"\nCutoff даты:")
print(f"  Train: до {train_end.date()} ({train_cutoff} дат)")
print(f"  Val: {unique_dates[train_cutoff].date()} - {val_end.date()} ({val_cutoff - train_cutoff} дат)")
print(f"  Test (OOT): {unique_dates[val_cutoff].date()}+ ({n_dates - val_cutoff} дат)")

# Создание split
train_df = df_sorted[df_sorted[DATE_COLUMN] <= train_end].copy()
val_df = df_sorted[(df_sorted[DATE_COLUMN] > train_end) & 
                   (df_sorted[DATE_COLUMN] <= val_end)].copy()
test_df = df_sorted[df_sorted[DATE_COLUMN] > val_end].copy()

# Stats
for name, df in [('TRAIN', train_df), ('VAL', val_df), ('TEST (OOT)', test_df)]:
    churn_r = df[TARGET_COLUMN].mean()
    print(f"\n{name}:")
    print(f"  Записей: {len(df):,}")
    print(f"  Churn rate: {churn_r:.4f} ({churn_r*100:.2f}%)")

print("\n✓ Temporal ordering verified - no data leakage")
print("═"*80)

---
## 4. PREPROCESSING PIPELINE

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# PREPROCESSING PIPELINE
# ═══════════════════════════════════════════════════════════════════════════════

class PreprocessingPipeline:
    """Preprocessing pipeline для моделей"""
    
    def __init__(self):
        self.fitted_columns = None
        self.final_features = None
        self.constant_cols = []
        self.outlier_bounds = {}
        self.numeric_imputer = None
        self.categorical_imputer = None
        self.numeric_cols_for_imputation = []
        self.categorical_cols_for_imputation = []
        self.features_to_drop_corr = []
    
    def fit_transform(self, train_df):
        """Fit and transform training data"""
        print("\n" + "═"*80)
        print("PREPROCESSING: FIT_TRANSFORM ON TRAIN")
        print("═"*80)
        
        df = train_df.copy()
        
        # Store columns
        self.fitted_columns = [c for c in df.columns 
                              if c not in ID_COLUMNS + [TARGET_COLUMN]]
        
        # 1. Remove constants
        df = self._remove_constants(df, fit=True)
        
        # 2. Handle outliers
        df = self._handle_outliers(df, fit=True)
        
        # 3. Handle missing
        df = self._handle_missing(df, fit=True)
        
        # 4. Remove correlations
        df = self._remove_correlations(df, fit=True)
        
        # Final features
        self.final_features = [c for c in df.columns 
                              if c not in ID_COLUMNS + [TARGET_COLUMN]]
        
        print(f"\n✓ Preprocessing complete")
        print(f"  Features: {len(self.final_features)}")
        
        return df
    
    def transform(self, df, dataset_name='test'):
        """Transform new data"""
        print(f"\nPreprocessing: {dataset_name}")
        df = df.copy()
        
        df = self._remove_constants(df, fit=False)
        df = self._handle_outliers(df, fit=False)
        df = self._handle_missing(df, fit=False)
        df = self._remove_correlations(df, fit=False)
        df = self._align_columns(df, dataset_name)
        
        print(f"  ✓ {dataset_name}: {df.shape}")
        return df
    
    def _remove_constants(self, df, fit):
        if fit:
            print("\n1. Removing constant columns...")
            for col in df.columns:
                if col in ID_COLUMNS + [TARGET_COLUMN]:
                    continue
                if df[col].nunique(dropna=False) == 1:
                    self.constant_cols.append(col)
            
            if self.constant_cols:
                df = df.drop(columns=self.constant_cols)
                print(f"   Removed: {len(self.constant_cols)}")
        return df
    
    def _handle_outliers(self, df, fit):
        if fit:
            print("\n2. Handling outliers...")
            keywords = ['profit', 'income', 'expense', 'margin', 'provision',
                       'balance', 'assets', 'liabilities']
            cols = [c for c in df.columns 
                   if any(kw in c.lower() for kw in keywords)
                   and c not in ID_COLUMNS + [TARGET_COLUMN] + CATEGORICAL_FEATURES]
            
            for col in cols:
                if df[col].dtype in ['float64', 'float32', 'int64', 'int32']:
                    Q1, Q3 = df[col].quantile([0.25, 0.75])
                    IQR = Q3 - Q1
                    self.outlier_bounds[col] = {
                        'lower': Q1 - OUTLIER_IQR_MULTIPLIER * IQR,
                        'upper': Q3 + OUTLIER_IQR_MULTIPLIER * IQR
                    }
            
            for col, bounds in self.outlier_bounds.items():
                df[col] = df[col].clip(lower=bounds['lower'], upper=bounds['upper'])
            
            print(f"   Clipped: {len(self.outlier_bounds)} columns")
        else:
            for col, bounds in self.outlier_bounds.items():
                if col in df.columns:
                    df[col] = df[col].clip(lower=bounds['lower'], upper=bounds['upper'])
        
        return df
    
    def _handle_missing(self, df, fit):
        if fit:
            print("\n3. Handling missing values...")
            self.numeric_cols_for_imputation = [
                c for c in df.select_dtypes(include=[np.number]).columns
                if c not in ID_COLUMNS + [TARGET_COLUMN]
            ]
            self.categorical_cols_for_imputation = [
                c for c in CATEGORICAL_FEATURES if c in df.columns
            ]
            
            self.numeric_imputer = SimpleImputer(strategy='median')
            self.categorical_imputer = SimpleImputer(strategy='most_frequent')
            
            if len(self.numeric_cols_for_imputation) > 0:
                df[self.numeric_cols_for_imputation] = self.numeric_imputer.fit_transform(
                    df[self.numeric_cols_for_imputation]
                )
            
            if len(self.categorical_cols_for_imputation) > 0:
                df[self.categorical_cols_for_imputation] = self.categorical_imputer.fit_transform(
                    df[self.categorical_cols_for_imputation]
                )
            
            print(f"   Imputed: {len(self.numeric_cols_for_imputation)} numeric, "
                  f"{len(self.categorical_cols_for_imputation)} categorical")
        else:
            if len(self.numeric_cols_for_imputation) > 0:
                present = [c for c in self.numeric_cols_for_imputation if c in df.columns]
                if present:
                    df[present] = self.numeric_imputer.transform(df[present])
            
            if len(self.categorical_cols_for_imputation) > 0:
                present = [c for c in self.categorical_cols_for_imputation if c in df.columns]
                if present:
                    df[present] = self.categorical_imputer.transform(df[present])
        
        return df
    
    def _remove_correlations(self, df, fit):
        if fit:
            print("\n4. Removing high correlations...")
            numeric = [c for c in df.select_dtypes(include=[np.number]).columns
                      if c not in ID_COLUMNS + [TARGET_COLUMN] + CATEGORICAL_FEATURES]
            
            if len(numeric) > 1:
                corr = df[numeric].corr().abs()
                upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
                self.features_to_drop_corr = [c for c in upper.columns
                                             if any(upper[c] > CORRELATION_THRESHOLD)]
                
                if self.features_to_drop_corr:
                    df = df.drop(columns=self.features_to_drop_corr)
                    print(f"   Removed: {len(self.features_to_drop_corr)}")
        
        return df
    
    def _align_columns(self, df, name):
        preserve = [c for c in ID_COLUMNS if c in df.columns]
        if TARGET_COLUMN in df.columns:
            preserve.append(TARGET_COLUMN)
        
        current = [c for c in df.columns if c not in preserve]
        missing = [c for c in self.final_features if c not in current]
        extra = [c for c in current if c not in self.final_features]
        
        if missing:
            for c in missing:
                df[c] = 0
        
        if extra:
            df = df.drop(columns=extra)
        
        order = preserve + self.final_features
        df = df[[c for c in order if c in df.columns]]
        
        return df


# Apply preprocessing
pipeline = PreprocessingPipeline()
train_processed = pipeline.fit_transform(train_df)
val_processed = pipeline.transform(val_df, 'validation')
test_processed = pipeline.transform(test_df, 'test (OOT)')

print("\n" + "═"*80)

---
## 5. РАЗДЕЛЕНИЕ ПО СЕГМЕНТАМ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# SEGMENT SPLIT
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("РАЗДЕЛЕНИЕ ПО СЕГМЕНТАМ")
print("═"*80)

# Segment 1: Small Business
seg1_train = train_processed[train_processed[SEGMENT_COLUMN].isin(SEGMENT_1_VALUES)].copy()
seg1_val = val_processed[val_processed[SEGMENT_COLUMN].isin(SEGMENT_1_VALUES)].copy()
seg1_test = test_processed[test_processed[SEGMENT_COLUMN].isin(SEGMENT_1_VALUES)].copy()

# Для seg1: УДАЛИТЬ segment_group (одно значение - бесполезен)
if SEGMENT_COLUMN in seg1_train.columns:
    seg1_train = seg1_train.drop(columns=[SEGMENT_COLUMN])
    seg1_val = seg1_val.drop(columns=[SEGMENT_COLUMN])
    seg1_test = seg1_test.drop(columns=[SEGMENT_COLUMN])
    print(f"\n✓ Для Segment 1: удален '{SEGMENT_COLUMN}' (одно значение)")

print(f"\nSegment 1 (SMALL_BUSINESS):")
print(f"  Train: {len(seg1_train):,} | Churn: {seg1_train[TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {len(seg1_val):,} | Churn: {seg1_val[TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {len(seg1_test):,} | Churn: {seg1_test[TARGET_COLUMN].mean()*100:.2f}%")

# Segment 2: Middle + Large Business
seg2_train = train_processed[train_processed[SEGMENT_COLUMN].isin(SEGMENT_2_VALUES)].copy()
seg2_val = val_processed[val_processed[SEGMENT_COLUMN].isin(SEGMENT_2_VALUES)].copy()
seg2_test = test_processed[test_processed[SEGMENT_COLUMN].isin(SEGMENT_2_VALUES)].copy()

# Для seg2: ОСТАВИТЬ segment_group (два значения - полезен)
print(f"\n✓ Для Segment 2: сохранен '{SEGMENT_COLUMN}' (два значения: {SEGMENT_2_VALUES})")

print(f"\nSegment 2 (MIDDLE + LARGE BUSINESS):")
print(f"  Train: {len(seg2_train):,} | Churn: {seg2_train[TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Val: {len(seg2_val):,} | Churn: {seg2_val[TARGET_COLUMN].mean()*100:.2f}%")
print(f"  Test: {len(seg2_test):,} | Churn: {seg2_test[TARGET_COLUMN].mean()*100:.2f}%")

print("═"*80)

---
## 6. ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЙ ПРИЗНАКОВ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ФУНКЦИЯ ДЛЯ ВИЗУАЛИЗАЦИИ РАСПРЕДЕЛЕНИЙ
# ═══════════════════════════════════════════════════════════════════════════════

def plot_feature_distributions(df, segment_name, color, save_path):
    """
    Визуализация распределений всех числовых признаков
    """
    # Выбираем только числовые признаки (без ID и target)
    exclude_cols = ID_COLUMNS + [TARGET_COLUMN]
    numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns 
                   if c not in exclude_cols]
    
    n_features = len(numeric_cols)
    
    # Определяем размер сетки
    n_cols = 10
    n_rows = (n_features + n_cols - 1) // n_cols
    
    print(f"\nВизуализация распределений: {segment_name}")
    print(f"  Признаков: {n_features}")
    print(f"  Grid: {n_rows}x{n_cols}")
    
    # Создаем большой график
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(40, 35))
    axes = axes.flatten()
    
    for idx, col in enumerate(tqdm(numeric_cols, desc=f"  Plotting {segment_name}")):
        ax = axes[idx]
        
        # Данные
        data = df[col].dropna()
        
        # Статистики
        nunique = df[col].nunique()
        mean_val = data.mean()
        median_val = data.median()
        skew_val = data.skew()
        
        # Histogram
        ax.hist(data, bins=50, color=color, alpha=0.7, edgecolor='black', linewidth=0.5)
        
        # Median line
        ax.axvline(median_val, color='red', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
        
        # Заголовок
        ax.set_title(f"{col}\nnunique={nunique}, mean={mean_val:.2f}, median={median_val:.2f}, skew={skew_val:.2f}",
                    fontsize=8)
        ax.tick_params(labelsize=6)
        ax.legend(fontsize=6)
    
    # Скрываем лишние оси
    for idx in range(n_features, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f"Распределения признаков: {segment_name}", fontsize=20, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(save_path, dpi=100, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Сохранено: {save_path}")
    
    return numeric_cols

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ВИЗУАЛИЗАЦИЯ: SEGMENT 1
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЙ ПРИЗНАКОВ")
print("═"*80)

seg1_numeric_cols = plot_feature_distributions(
    seg1_train, 
    "Segment 1 (SMALL_BUSINESS)",
    'steelblue',
    FIGURES_DIR / 'distributions_segment1.png'
)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ВИЗУАЛИЗАЦИЯ: SEGMENT 2
# ═══════════════════════════════════════════════════════════════════════════════

seg2_numeric_cols = plot_feature_distributions(
    seg2_train, 
    "Segment 2 (MIDDLE + LARGE BUSINESS)",
    'seagreen',
    FIGURES_DIR / 'distributions_segment2.png'
)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# СТАТИСТИКИ ПРИЗНАКОВ
# ═══════════════════════════════════════════════════════════════════════════════

def create_feature_statistics(df, numeric_cols, save_path):
    """
    Создает таблицу со статистиками признаков
    """
    stats_list = []
    
    for col in numeric_cols:
        data = df[col].dropna()
        
        stats_list.append({
            'feature_name': col,
            'nunique': df[col].nunique(),
            'min': data.min(),
            'max': data.max(),
            'mean': data.mean(),
            'median': data.median(),
            'std': data.std(),
            'skewness': data.skew(),
            'kurtosis': data.kurtosis()
        })
    
    stats_df = pd.DataFrame(stats_list)
    stats_df = stats_df.sort_values('skewness', key=abs, ascending=False).reset_index(drop=True)
    stats_df.to_csv(save_path, index=False)
    
    print(f"\n✓ Статистики сохранены: {save_path}")
    print(f"\nТоп-10 признаков с наибольшей асимметрией:")
    print(stats_df[['feature_name', 'skewness', 'mean', 'median']].head(10).to_string(index=False))
    
    return stats_df


# Segment 1
print("\n" + "─"*80)
print("СТАТИСТИКИ: SEGMENT 1")
print("─"*80)
seg1_stats = create_feature_statistics(
    seg1_train, seg1_numeric_cols, 
    OUTPUT_DIR / 'feature_statistics_seg1.csv'
)

# Segment 2
print("\n" + "─"*80)
print("СТАТИСТИКИ: SEGMENT 2")
print("─"*80)
seg2_stats = create_feature_statistics(
    seg2_train, seg2_numeric_cols, 
    OUTPUT_DIR / 'feature_statistics_seg2.csv'
)

print("\n" + "═"*80)

---
## 7. КАСТОМНЫЙ K-MEANS UNDERSAMPLING

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ФУНКЦИЯ: KMEANS UNDERSAMPLING
# ═══════════════════════════════════════════════════════════════════════════════

def kmeans_undersampling(X_train, y_train, target_ratio=20, n_clusters=30, random_state=42):
    """
    Умный undersampling через k-means кластеризацию класса 0
    
    Parameters:
    -----------
    X_train : DataFrame - обучающие данные
    y_train : Series - таргет
    target_ratio : int - желаемое соотношение class_0 / class_1 (по умолчанию 20)
    n_clusters : int - количество кластеров для класса 0 (по умолчанию 30)
    random_state : int - для воспроизводимости
    
    Returns:
    --------
    X_balanced : DataFrame - сбалансированные данные
    y_balanced : Series - сбалансированный таргет
    cluster_info : dict - информация о кластерах
    """
    
    print("\n" + "─"*80)
    print("K-MEANS UNDERSAMPLING")
    print("─"*80)
    
    # Шаг 1: Разделение по классам
    X_class_0 = X_train[y_train == 0].copy()
    X_class_1 = X_train[y_train == 1].copy()
    y_class_0 = y_train[y_train == 0].copy()
    y_class_1 = y_train[y_train == 1].copy()
    
    n_class_1 = len(X_class_1)
    n_class_0_original = len(X_class_0)
    
    print(f"\nИсходное распределение:")
    print(f"  Class 0 (не ушли): {n_class_0_original:,}")
    print(f"  Class 1 (ушли): {n_class_1:,}")
    print(f"  Соотношение: 1:{n_class_0_original/n_class_1:.1f}")
    
    # Шаг 2: Расчет целевого количества
    n_class_0_target = int(n_class_1 * target_ratio)
    
    print(f"\nЦелевое распределение:")
    print(f"  Class 0 (целевое): {n_class_0_target:,}")
    print(f"  Class 1: {n_class_1:,}")
    print(f"  Целевое соотношение: 1:{target_ratio}")
    
    # Шаг 3: Стандартизация признаков
    print(f"\nШаг 1: Стандартизация признаков...")
    scaler = StandardScaler()
    X_class_0_scaled = scaler.fit_transform(X_class_0)
    print(f"  ✓ Признаки стандартизированы (важно для k-means)")
    
    # Шаг 4: K-Means кластеризация класса 0
    print(f"\nШаг 2: K-Means кластеризация класса 0...")
    kmeans = MiniBatchKMeans(
        n_clusters=n_clusters,
        batch_size=KMEANS_BATCH_SIZE,
        random_state=random_state,
        verbose=0,
        n_init=3
    )
    cluster_labels = kmeans.fit_predict(X_class_0_scaled)
    print(f"  ✓ K-Means кластеризация выполнена")
    print(f"  Количество кластеров: {n_clusters}")
    
    # Шаг 5: Анализ размеров кластеров
    cluster_sizes = pd.Series(cluster_labels).value_counts().sort_index()
    samples_per_cluster = n_class_0_target // n_clusters
    
    print(f"  Среднее желаемое из кластера: {samples_per_cluster}")
    print(f"\nРаспределение по кластерам:")
    print(f"  Min размер: {cluster_sizes.min()}")
    print(f"  Max размер: {cluster_sizes.max()}")
    print(f"  Mean размер: {cluster_sizes.mean():.0f}")
    
    # Шаг 6: УМНЫЙ SAMPLING из кластеров
    print(f"\nШаг 3: Умный sampling из кластеров...")
    
    sampled_indices = []
    small_clusters_count = 0
    
    # Первый проход
    for cluster_id in range(n_clusters):
        cluster_mask = (cluster_labels == cluster_id)
        cluster_indices = np.where(cluster_mask)[0]
        cluster_size = len(cluster_indices)
        
        if cluster_size <= samples_per_cluster:
            # МАЛЕНЬКИЙ кластер - берем ВСЕ
            selected_indices = cluster_indices
            small_clusters_count += 1
        else:
            # БОЛЬШОЙ кластер - берем samples_per_cluster случайно
            selected_indices = np.random.choice(
                cluster_indices, 
                size=samples_per_cluster, 
                replace=False
            )
        
        sampled_indices.extend(selected_indices)
    
    n_sampled = len(sampled_indices)
    
    print(f"  Первый проход sampling:")
    print(f"    Маленьких кластеров (взяты полностью): {small_clusters_count} из {n_clusters}")
    print(f"    Набрано примеров: {n_sampled:,} / {n_class_0_target:,}")
    
    # Если не хватает - добираем из больших кластеров
    if n_sampled < n_class_0_target:
        shortage = n_class_0_target - n_sampled
        print(f"    Нехватка: {shortage:,}")
        
        # Находим большие кластеры
        large_clusters = []
        for cluster_id in range(n_clusters):
            cluster_size = (cluster_labels == cluster_id).sum()
            if cluster_size > samples_per_cluster:
                large_clusters.append(cluster_id)
        
        # Распределяем нехватку пропорционально
        additional_per_cluster = shortage // len(large_clusters) + 1
        
        for cluster_id in large_clusters:
            if len(sampled_indices) >= n_class_0_target:
                break
            
            cluster_mask = (cluster_labels == cluster_id)
            cluster_indices = np.where(cluster_mask)[0]
            
            # Исключаем уже взятые
            available = np.setdiff1d(cluster_indices, sampled_indices)
            
            n_to_take = min(additional_per_cluster, len(available), 
                           n_class_0_target - len(sampled_indices))
            
            if n_to_take > 0:
                additional = np.random.choice(available, size=n_to_take, replace=False)
                sampled_indices.extend(additional)
        
        print(f"    Добрано дополнительно: {len(sampled_indices) - n_sampled}")
    
    # Финальный набор
    sampled_indices = np.array(sampled_indices)[:n_class_0_target]
    
    print(f"\nИтого после умного sampling:")
    print(f"  Class 0 отобрано: {len(sampled_indices):,}")
    print(f"  Reduction: {(1 - len(sampled_indices)/n_class_0_original)*100:.1f}%")
    
    # Шаг 7: Формирование финальной выборки
    print(f"\nШаг 4: Формирование финальной выборки...")
    
    # Отобранные примеры класса 0
    X_class_0_sampled = X_class_0.iloc[sampled_indices].copy()
    y_class_0_sampled = y_class_0.iloc[sampled_indices].copy()
    
    # Объединяем с классом 1 (весь)
    X_balanced = pd.concat([X_class_0_sampled, X_class_1], ignore_index=True)
    y_balanced = pd.concat([y_class_0_sampled, y_class_1], ignore_index=True)
    
    # SHUFFLE (важно!)
    shuffle_idx = np.random.RandomState(random_state).permutation(len(X_balanced))
    X_balanced = X_balanced.iloc[shuffle_idx].reset_index(drop=True)
    y_balanced = y_balanced.iloc[shuffle_idx].reset_index(drop=True)
    
    # Финальная проверка
    final_ratio = (y_balanced == 0).sum() / (y_balanced == 1).sum()
    print(f"\nФинальное распределение:")
    print(f"  Class 0: {(y_balanced == 0).sum():,}")
    print(f"  Class 1: {(y_balanced == 1).sum():,}")
    print(f"  Итоговое соотношение: 1:{final_ratio:.1f}")
    print(f"  ✓ Данные перемешаны (shuffle)")
    
    # Информация о кластерах
    cluster_info = {
        'n_clusters': n_clusters,
        'small_clusters_count': small_clusters_count,
        'cluster_sizes': cluster_sizes.to_dict(),
        'samples_per_cluster_target': samples_per_cluster,
        'reduction_percent': (1 - len(sampled_indices)/n_class_0_original)*100,
        'original_class_0': n_class_0_original,
        'sampled_class_0': len(sampled_indices),
        'class_1': n_class_1,
        'final_ratio': final_ratio
    }
    
    print("─"*80)
    
    # Очистка памяти
    del X_class_0_scaled, cluster_labels
    gc.collect()
    
    return X_balanced, y_balanced, cluster_info

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ПРИМЕНЕНИЕ K-MEANS UNDERSAMPLING: SEGMENT 1
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("K-MEANS UNDERSAMPLING: SEGMENT 1 (SMALL BUSINESS)")
print("═"*80)

seg1_train_balanced, seg1_y_train_balanced, seg1_cluster_info = kmeans_undersampling(
    X_train=seg1_train.drop(columns=[TARGET_COLUMN]),
    y_train=seg1_train[TARGET_COLUMN],
    target_ratio=TARGET_RATIO,
    n_clusters=N_CLUSTERS,
    random_state=RANDOM_SEED
)

# Сохранение информации о кластерах
with open(OUTPUT_DIR / 'cluster_info_seg1.json', 'w') as f:
    json.dump(seg1_cluster_info, f, indent=2)
print(f"\n✓ Информация о кластерах сохранена: {OUTPUT_DIR / 'cluster_info_seg1.json'}")

print("═"*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ПРИМЕНЕНИЕ K-MEANS UNDERSAMPLING: SEGMENT 2
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("K-MEANS UNDERSAMPLING: SEGMENT 2 (MIDDLE + LARGE BUSINESS)")
print("═"*80)

seg2_train_balanced, seg2_y_train_balanced, seg2_cluster_info = kmeans_undersampling(
    X_train=seg2_train.drop(columns=[TARGET_COLUMN]),
    y_train=seg2_train[TARGET_COLUMN],
    target_ratio=TARGET_RATIO,
    n_clusters=N_CLUSTERS,
    random_state=RANDOM_SEED
)

# Сохранение информации о кластерах
with open(OUTPUT_DIR / 'cluster_info_seg2.json', 'w') as f:
    json.dump(seg2_cluster_info, f, indent=2)
print(f"\n✓ Информация о кластерах сохранена: {OUTPUT_DIR / 'cluster_info_seg2.json'}")

print("═"*80)

---
## 8. ОБУЧЕНИЕ МОДЕЛЕЙ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# HELPER FUNCTIONS ДЛЯ ОБУЧЕНИЯ
# ═══════════════════════════════════════════════════════════════════════════════

def find_optimal_threshold(y_true, y_pred_proba):
    """Поиск оптимального порога по F1"""
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        score = f1_score(y_true, y_pred)
        scores.append(score)
    
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]


def calculate_metrics(y_true, y_pred_proba, optimal_threshold):
    """Расчет всех метрик"""
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)
    
    metrics = {
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'pr_auc': average_precision_score(y_true, y_pred_proba),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'accuracy': accuracy_score(y_true, y_pred),
    }
    metrics['gini'] = 2 * metrics['roc_auc'] - 1
    
    cm = confusion_matrix(y_true, y_pred)
    metrics['tn'] = cm[0, 0]
    metrics['fp'] = cm[0, 1]
    metrics['fn'] = cm[1, 0]
    metrics['tp'] = cm[1, 1]
    
    return metrics

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ОБУЧЕНИЕ МОДЕЛЕЙ: ОСНОВНАЯ ФУНКЦИЯ
# ═══════════════════════════════════════════════════════════════════════════════

def train_all_models(seg_name, seg_train_balanced, seg_y_train_balanced, 
                     seg_val, seg_test, categorical_features):
    """
    Обучение 3 алгоритмов: CatBoost, LightGBM, XGBoost
    """
    results = []
    trained_models = {}
    predictions = {}
    
    # Подготовка данных
    X_train = seg_train_balanced
    y_train = seg_y_train_balanced
    
    X_val = seg_val.drop(columns=[TARGET_COLUMN])
    y_val = seg_val[TARGET_COLUMN]
    
    X_test = seg_test.drop(columns=[TARGET_COLUMN])
    y_test = seg_test[TARGET_COLUMN]
    
    print(f"\nДанные:")
    print(f"  Train (balanced): {X_train.shape}")
    print(f"  Val (original): {X_val.shape}")
    print(f"  Test (original): {X_test.shape}")
    
    # Категориальные признаки (фильтруем те, которые есть)
    cat_features_present = [c for c in categorical_features if c in X_train.columns]
    print(f"  Категориальные: {cat_features_present}")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # 1. CATBOOST
    # ─────────────────────────────────────────────────────────────────────────────
    
    print("\n" + "─"*80)
    print("ОБУЧЕНИЕ: CatBoost")
    print("─"*80)
    
    start_time = time.time()
    
    # Подготовка данных для CatBoost
    X_train_cat = X_train.copy()
    X_val_cat = X_val.copy()
    X_test_cat = X_test.copy()
    
    for cat in cat_features_present:
        X_train_cat[cat] = X_train_cat[cat].astype(str)
        X_val_cat[cat] = X_val_cat[cat].astype(str)
        X_test_cat[cat] = X_test_cat[cat].astype(str)
    
    cat_indices = [i for i, c in enumerate(X_train_cat.columns) if c in cat_features_present]
    
    train_pool = Pool(X_train_cat, y_train, cat_features=cat_indices)
    val_pool = Pool(X_val_cat, y_val, cat_features=cat_indices)
    test_pool = Pool(X_test_cat, y_test, cat_features=cat_indices)
    
    catboost_model = CatBoostClassifier(
        iterations=500,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        early_stopping_rounds=50,
        verbose=100,
        random_seed=RANDOM_SEED,
        task_type='CPU'
    )
    
    catboost_model.fit(train_pool, eval_set=val_pool, plot=False)
    
    train_time = time.time() - start_time
    
    # Предсказания
    y_val_proba_cb = catboost_model.predict_proba(val_pool)[:, 1]
    y_test_proba_cb = catboost_model.predict_proba(test_pool)[:, 1]
    
    # Optimal threshold
    optimal_thresh_cb, _ = find_optimal_threshold(y_val, y_val_proba_cb)
    print(f"\n  Optimal threshold: {optimal_thresh_cb:.3f}")
    
    # Metrics on test
    metrics_cb = calculate_metrics(y_test, y_test_proba_cb, optimal_thresh_cb)
    
    results.append({
        'segment_group': seg_name,
        'algorithm': 'CatBoost',
        'roc_auc': metrics_cb['roc_auc'],
        'gini': metrics_cb['gini'],
        'pr_auc': metrics_cb['pr_auc'],
        'f1': metrics_cb['f1'],
        'precision': metrics_cb['precision'],
        'recall': metrics_cb['recall'],
        'accuracy': metrics_cb['accuracy'],
        'optimal_threshold': optimal_thresh_cb,
        'tn': metrics_cb['tn'],
        'fp': metrics_cb['fp'],
        'fn': metrics_cb['fn'],
        'tp': metrics_cb['tp'],
        'train_time_sec': train_time
    })
    
    trained_models['CatBoost'] = catboost_model
    predictions['CatBoost'] = {'val': y_val_proba_cb, 'test': y_test_proba_cb}
    
    print(f"\n  Test ROC-AUC: {metrics_cb['roc_auc']:.4f} | Gini: {metrics_cb['gini']:.4f} | F1: {metrics_cb['f1']:.4f}")
    print(f"  Train time: {train_time:.1f}s")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # 2. LIGHTGBM
    # ─────────────────────────────────────────────────────────────────────────────
    
    print("\n" + "─"*80)
    print("ОБУЧЕНИЕ: LightGBM")
    print("─"*80)
    
    start_time = time.time()
    
    # Подготовка данных для LightGBM
    X_train_lgb = X_train.copy()
    X_val_lgb = X_val.copy()
    X_test_lgb = X_test.copy()
    
    for cat in cat_features_present:
        X_train_lgb[cat] = X_train_lgb[cat].astype('category')
        X_val_lgb[cat] = X_val_lgb[cat].astype('category')
        X_test_lgb[cat] = X_test_lgb[cat].astype('category')
    
    lgb_model = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        num_leaves=31,
        min_child_samples=100,
        random_state=RANDOM_SEED,
        verbose=-1
    )
    
    lgb_model.fit(
        X_train_lgb, y_train,
        eval_set=[(X_val_lgb, y_val)],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    train_time = time.time() - start_time
    
    # Предсказания
    y_val_proba_lgb = lgb_model.predict_proba(X_val_lgb)[:, 1]
    y_test_proba_lgb = lgb_model.predict_proba(X_test_lgb)[:, 1]
    
    # Optimal threshold
    optimal_thresh_lgb, _ = find_optimal_threshold(y_val, y_val_proba_lgb)
    print(f"\n  Optimal threshold: {optimal_thresh_lgb:.3f}")
    
    # Metrics on test
    metrics_lgb = calculate_metrics(y_test, y_test_proba_lgb, optimal_thresh_lgb)
    
    results.append({
        'segment_group': seg_name,
        'algorithm': 'LightGBM',
        'roc_auc': metrics_lgb['roc_auc'],
        'gini': metrics_lgb['gini'],
        'pr_auc': metrics_lgb['pr_auc'],
        'f1': metrics_lgb['f1'],
        'precision': metrics_lgb['precision'],
        'recall': metrics_lgb['recall'],
        'accuracy': metrics_lgb['accuracy'],
        'optimal_threshold': optimal_thresh_lgb,
        'tn': metrics_lgb['tn'],
        'fp': metrics_lgb['fp'],
        'fn': metrics_lgb['fn'],
        'tp': metrics_lgb['tp'],
        'train_time_sec': train_time
    })
    
    trained_models['LightGBM'] = lgb_model
    predictions['LightGBM'] = {'val': y_val_proba_lgb, 'test': y_test_proba_lgb}
    
    print(f"\n  Test ROC-AUC: {metrics_lgb['roc_auc']:.4f} | Gini: {metrics_lgb['gini']:.4f} | F1: {metrics_lgb['f1']:.4f}")
    print(f"  Train time: {train_time:.1f}s")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # 3. XGBOOST
    # ─────────────────────────────────────────────────────────────────────────────
    
    print("\n" + "─"*80)
    print("ОБУЧЕНИЕ: XGBoost")
    print("─"*80)
    
    start_time = time.time()
    
    # Подготовка данных для XGBoost (LabelEncoder для категориальных)
    X_train_xgb = X_train.copy()
    X_val_xgb = X_val.copy()
    X_test_xgb = X_test.copy()
    
    label_encoders = {}
    for cat in cat_features_present:
        le = LabelEncoder()
        X_train_xgb[cat] = le.fit_transform(X_train_xgb[cat].astype(str))
        X_val_xgb[cat] = le.transform(X_val_xgb[cat].astype(str))
        X_test_xgb[cat] = le.transform(X_test_xgb[cat].astype(str))
        label_encoders[cat] = le
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        min_child_weight=100,
        random_state=RANDOM_SEED,
        eval_metric='auc',
        tree_method='hist',
        early_stopping_rounds=50
    )
    
    xgb_model.fit(
        X_train_xgb, y_train,
        eval_set=[(X_val_xgb, y_val)],
        verbose=100
    )
    
    train_time = time.time() - start_time
    
    # Предсказания
    y_val_proba_xgb = xgb_model.predict_proba(X_val_xgb)[:, 1]
    y_test_proba_xgb = xgb_model.predict_proba(X_test_xgb)[:, 1]
    
    # Optimal threshold
    optimal_thresh_xgb, _ = find_optimal_threshold(y_val, y_val_proba_xgb)
    print(f"\n  Optimal threshold: {optimal_thresh_xgb:.3f}")
    
    # Metrics on test
    metrics_xgb = calculate_metrics(y_test, y_test_proba_xgb, optimal_thresh_xgb)
    
    results.append({
        'segment_group': seg_name,
        'algorithm': 'XGBoost',
        'roc_auc': metrics_xgb['roc_auc'],
        'gini': metrics_xgb['gini'],
        'pr_auc': metrics_xgb['pr_auc'],
        'f1': metrics_xgb['f1'],
        'precision': metrics_xgb['precision'],
        'recall': metrics_xgb['recall'],
        'accuracy': metrics_xgb['accuracy'],
        'optimal_threshold': optimal_thresh_xgb,
        'tn': metrics_xgb['tn'],
        'fp': metrics_xgb['fp'],
        'fn': metrics_xgb['fn'],
        'tp': metrics_xgb['tp'],
        'train_time_sec': train_time
    })
    
    trained_models['XGBoost'] = xgb_model
    predictions['XGBoost'] = {'val': y_val_proba_xgb, 'test': y_test_proba_xgb}
    
    print(f"\n  Test ROC-AUC: {metrics_xgb['roc_auc']:.4f} | Gini: {metrics_xgb['gini']:.4f} | F1: {metrics_xgb['f1']:.4f}")
    print(f"  Train time: {train_time:.1f}s")
    
    # Очистка памяти
    gc.collect()
    
    return results, trained_models, predictions, y_val, y_test

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ОБУЧЕНИЕ: SEGMENT 1
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("ОБУЧЕНИЕ МОДЕЛЕЙ: SEGMENT 1")
print("═"*80)

# Для seg1 категориальные без segment_group
seg1_cat_features = [c for c in CATEGORICAL_FEATURES if c != SEGMENT_COLUMN]

seg1_results, seg1_models, seg1_predictions, seg1_y_val, seg1_y_test = train_all_models(
    seg_name='Segment 1 (SMALL_BUSINESS)',
    seg_train_balanced=seg1_train_balanced,
    seg_y_train_balanced=seg1_y_train_balanced,
    seg_val=seg1_val,
    seg_test=seg1_test,
    categorical_features=seg1_cat_features
)

print("\n" + "═"*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ОБУЧЕНИЕ: SEGMENT 2
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("ОБУЧЕНИЕ МОДЕЛЕЙ: SEGMENT 2")
print("═"*80)

# Для seg2 категориальные включают segment_group
seg2_cat_features = CATEGORICAL_FEATURES

seg2_results, seg2_models, seg2_predictions, seg2_y_val, seg2_y_test = train_all_models(
    seg_name='Segment 2 (MIDDLE + LARGE)',
    seg_train_balanced=seg2_train_balanced,
    seg_y_train_balanced=seg2_y_train_balanced,
    seg_val=seg2_val,
    seg_test=seg2_test,
    categorical_features=seg2_cat_features
)

print("\n" + "═"*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
print("═"*80)

# Объединяем результаты
all_results = seg1_results + seg2_results
results_df = pd.DataFrame(all_results)

# Сортируем
results_df = results_df.sort_values(['segment_group', 'roc_auc'], ascending=[True, False])

# Сохраняем
results_df.to_csv(OUTPUT_DIR / 'kmeans_undersampling_results.csv', index=False)
print(f"\n✓ Результаты сохранены: {OUTPUT_DIR / 'kmeans_undersampling_results.csv'}")

# Выводим таблицу
print("\n" + results_df.to_string(index=False))

print("\n" + "═"*80)

---
## 9. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ROC CURVES: SEGMENT 1
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("ВИЗУАЛИЗАЦИЯ: ROC CURVES")
print("═"*80)

fig, ax = plt.subplots(figsize=(10, 8))

colors = {'CatBoost': 'blue', 'LightGBM': 'green', 'XGBoost': 'red'}

for algo in ['CatBoost', 'LightGBM', 'XGBoost']:
    y_proba = seg1_predictions[algo]['test']
    fpr, tpr, _ = roc_curve(seg1_y_test, y_proba)
    auc_score = roc_auc_score(seg1_y_test, y_proba)
    
    ax.plot(fpr, tpr, linewidth=2, label=f'{algo} (AUC={auc_score:.4f})', color=colors[algo])

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves: Segment 1 (SMALL_BUSINESS)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'roc_segment1_kmeans.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Сохранено: {FIGURES_DIR / 'roc_segment1_kmeans.png'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ROC CURVES: SEGMENT 2
# ═══════════════════════════════════════════════════════════════════════════════

fig, ax = plt.subplots(figsize=(10, 8))

for algo in ['CatBoost', 'LightGBM', 'XGBoost']:
    y_proba = seg2_predictions[algo]['test']
    fpr, tpr, _ = roc_curve(seg2_y_test, y_proba)
    auc_score = roc_auc_score(seg2_y_test, y_proba)
    
    ax.plot(fpr, tpr, linewidth=2, label=f'{algo} (AUC={auc_score:.4f})', color=colors[algo])

ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curves: Segment 2 (MIDDLE + LARGE BUSINESS)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'roc_segment2_kmeans.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Сохранено: {FIGURES_DIR / 'roc_segment2_kmeans.png'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# BARPLOT СРАВНЕНИЕ МЕТРИК
# ═══════════════════════════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Segment 1
seg1_data = results_df[results_df['segment_group'].str.contains('Segment 1')]
ax1 = axes[0]
x_pos = np.arange(len(seg1_data))
bars1 = ax1.bar(x_pos, seg1_data['roc_auc'], 
                color=['blue', 'green', 'red'], alpha=0.7, edgecolor='black')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(seg1_data['algorithm'])
ax1.set_ylabel('ROC-AUC', fontsize=12)
ax1.set_title('Segment 1 (SMALL_BUSINESS)', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 1)
ax1.grid(axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars1, seg1_data['roc_auc'])):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 0.02, 
            f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Segment 2
seg2_data = results_df[results_df['segment_group'].str.contains('Segment 2')]
ax2 = axes[1]
x_pos = np.arange(len(seg2_data))
bars2 = ax2.bar(x_pos, seg2_data['roc_auc'], 
                color=['blue', 'green', 'red'], alpha=0.7, edgecolor='black')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(seg2_data['algorithm'])
ax2.set_ylabel('ROC-AUC', fontsize=12)
ax2.set_title('Segment 2 (MIDDLE + LARGE)', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 1)
ax2.grid(axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars2, seg2_data['roc_auc'])):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.02, 
            f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.suptitle('Сравнение алгоритмов: ROC-AUC (Test OOT)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'metrics_comparison_kmeans.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Сохранено: {FIGURES_DIR / 'metrics_comparison_kmeans.png'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# FEATURE IMPORTANCE: ЛУЧШИЕ МОДЕЛИ
# ═══════════════════════════════════════════════════════════════════════════════

def plot_feature_importance(model, model_name, segment_name, save_path, top_n=20):
    """
    Визуализация feature importance
    """
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        feature_names = model.feature_names_in_ if hasattr(model, 'feature_names_in_') else None
    elif hasattr(model, 'get_feature_importance'):
        importances = model.get_feature_importance()
        feature_names = model.feature_names_
    else:
        print(f"  Модель {model_name} не поддерживает feature importance")
        return
    
    # Создаем DataFrame
    if feature_names is None:
        feature_names = [f'feature_{i}' for i in range(len(importances))]
    
    fi_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    })
    fi_df = fi_df.sort_values('importance', ascending=False).head(top_n)
    
    # График
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(range(len(fi_df)), fi_df['importance'], color='steelblue', edgecolor='black')
    ax.set_yticks(range(len(fi_df)))
    ax.set_yticklabels(fi_df['feature'])
    ax.invert_yaxis()
    ax.set_xlabel('Importance', fontsize=12)
    ax.set_title(f'Top-{top_n} Feature Importance: {model_name}\n{segment_name}', 
                fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Сохранено: {save_path}")


print("\n" + "═"*80)
print("FEATURE IMPORTANCE")
print("═"*80)

# Лучшая модель для Segment 1
best_seg1 = seg1_data.iloc[0]
best_model_seg1 = seg1_models[best_seg1['algorithm']]
print(f"\nЛучшая модель Segment 1: {best_seg1['algorithm']} (ROC-AUC: {best_seg1['roc_auc']:.4f})")
plot_feature_importance(
    best_model_seg1, 
    best_seg1['algorithm'], 
    'Segment 1 (SMALL_BUSINESS)',
    FIGURES_DIR / 'feature_importance_seg1_kmeans.png'
)

# Лучшая модель для Segment 2
best_seg2 = seg2_data.iloc[0]
best_model_seg2 = seg2_models[best_seg2['algorithm']]
print(f"\nЛучшая модель Segment 2: {best_seg2['algorithm']} (ROC-AUC: {best_seg2['roc_auc']:.4f})")
plot_feature_importance(
    best_model_seg2, 
    best_seg2['algorithm'], 
    'Segment 2 (MIDDLE + LARGE)',
    FIGURES_DIR / 'feature_importance_seg2_kmeans.png'
)

print("\n" + "═"*80)

---
## 10. ФИНАЛЬНЫЙ ОТЧЕТ

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ФИНАЛЬНЫЙ ОТЧЕТ
# ═══════════════════════════════════════════════════════════════════════════════

report = []
report.append("═"*80)
report.append("РЕЗУЛЬТАТЫ: K-MEANS UNDERSAMPLING (30 кластеров, соотношение 1:20)")
report.append("═"*80)
report.append("")
report.append(f"Дата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report.append(f"Random seed: {RANDOM_SEED}")
report.append("")

# SEGMENT 1
report.append("SEGMENT 1: SMALL BUSINESS")
report.append("─"*80)
report.append("")
report.append("Undersampling эффект:")
report.append(f"  Исходно: {seg1_cluster_info['original_class_0']:,} примеров класса 0")
report.append(f"  После: {seg1_cluster_info['sampled_class_0']:,} примеров класса 0")
report.append(f"  Reduction: {seg1_cluster_info['reduction_percent']:.1f}%")
report.append(f"  Маленьких кластеров сохранено полностью: {seg1_cluster_info['small_clusters_count']} из {N_CLUSTERS}")
report.append("")
report.append("Результаты моделей (Test OOT):")
for _, row in seg1_data.iterrows():
    report.append(f"  {row['algorithm']:<12}: ROC-AUC = {row['roc_auc']:.4f} | Gini = {row['gini']:.4f} | F1 = {row['f1']:.4f} | Time = {row['train_time_sec']:.0f}s")
report.append("")
report.append(f"Лучшая модель: {best_seg1['algorithm']} (ROC-AUC: {best_seg1['roc_auc']:.4f})")
report.append(f"Optimal threshold: {best_seg1['optimal_threshold']:.3f}")
report.append("")

# SEGMENT 2
report.append("SEGMENT 2: MIDDLE + LARGE BUSINESS")
report.append("─"*80)
report.append("")
report.append("Undersampling эффект:")
report.append(f"  Исходно: {seg2_cluster_info['original_class_0']:,} примеров класса 0")
report.append(f"  После: {seg2_cluster_info['sampled_class_0']:,} примеров класса 0")
report.append(f"  Reduction: {seg2_cluster_info['reduction_percent']:.1f}%")
report.append(f"  Маленьких кластеров сохранено полностью: {seg2_cluster_info['small_clusters_count']} из {N_CLUSTERS}")
report.append("")
report.append("Результаты моделей (Test OOT):")
for _, row in seg2_data.iterrows():
    report.append(f"  {row['algorithm']:<12}: ROC-AUC = {row['roc_auc']:.4f} | Gini = {row['gini']:.4f} | F1 = {row['f1']:.4f} | Time = {row['train_time_sec']:.0f}s")
report.append("")
report.append(f"Лучшая модель: {best_seg2['algorithm']} (ROC-AUC: {best_seg2['roc_auc']:.4f})")
report.append(f"Optimal threshold: {best_seg2['optimal_threshold']:.3f}")
report.append("")

# ОБЩИЕ ВЫВОДЫ
report.append("ОБЩИЕ ВЫВОДЫ")
report.append("─"*80)
report.append("")
report.append("1. Эффективность K-Means undersampling:")
report.append(f"   - Умное сохранение маленьких кластеров позволило сохранить разнообразие данных")
report.append(f"   - Reduction класса 0: Segment 1 = {seg1_cluster_info['reduction_percent']:.1f}%, Segment 2 = {seg2_cluster_info['reduction_percent']:.1f}%")
report.append(f"   - Достигнуто целевое соотношение 1:{TARGET_RATIO} для обеих групп")
report.append("")
report.append("2. Сравнение алгоритмов:")
report.append(f"   - Лучший для Segment 1: {best_seg1['algorithm']} (ROC-AUC: {best_seg1['roc_auc']:.4f})")
report.append(f"   - Лучший для Segment 2: {best_seg2['algorithm']} (ROC-AUC: {best_seg2['roc_auc']:.4f})")
avg_roc_auc = results_df.groupby('algorithm')['roc_auc'].mean().sort_values(ascending=False)
report.append(f"   - В среднем лучший: {avg_roc_auc.index[0]} (средний ROC-AUC: {avg_roc_auc.values[0]:.4f})")
report.append("")
report.append("3. Рекомендации:")
report.append(f"   - Использовать {best_seg1['algorithm']} для Segment 1 (SMALL_BUSINESS)")
report.append(f"   - Использовать {best_seg2['algorithm']} для Segment 2 (MIDDLE + LARGE)")
report.append(f"   - K-Means undersampling с {N_CLUSTERS} кластерами показал отличные результаты")
report.append(f"   - Умная стратегия sampling превосходит простой random undersampling")
report.append("")
report.append("═"*80)

# Выводим отчет
report_text = "\n".join(report)
print(report_text)

# Сохраняем отчет
with open(OUTPUT_DIR / 'final_report_kmeans.txt', 'w', encoding='utf-8') as f:
    f.write(report_text)

print(f"\n✓ Отчет сохранен: {OUTPUT_DIR / 'final_report_kmeans.txt'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ФИНАЛ
# ═══════════════════════════════════════════════════════════════════════════════

print("\n\n" + "═"*80)
print("✓✓✓ ЭКСПЕРИМЕНТЫ ЗАВЕРШЕНЫ УСПЕШНО ✓✓✓")
print("═"*80)
print("\nСозданные файлы:")
print("\n1. Визуализации распределений:")
print("   - figures/distributions_segment1.png")
print("   - figures/distributions_segment2.png")
print("\n2. Статистики признаков:")
print("   - output/feature_statistics_seg1.csv")
print("   - output/feature_statistics_seg2.csv")
print("\n3. Информация о кластерах:")
print("   - output/cluster_info_seg1.json")
print("   - output/cluster_info_seg2.json")
print("\n4. Результаты моделей:")
print("   - output/kmeans_undersampling_results.csv")
print("\n5. Визуализации результатов:")
print("   - figures/roc_segment1_kmeans.png")
print("   - figures/roc_segment2_kmeans.png")
print("   - figures/metrics_comparison_kmeans.png")
print("   - figures/feature_importance_seg1_kmeans.png")
print("   - figures/feature_importance_seg2_kmeans.png")
print("\n6. Финальный отчет:")
print("   - output/final_report_kmeans.txt")
print("\n" + "═"*80)
print("Воспроизводимость: Run All → идентичные результаты (random_seed=42)")
print("═"*80)